In [1]:
import pandas as pd
from datetime import datetime
import re
import numpy as np

# Read and clean raw files

This notebook prepares the xlsx files (downloaded from ABS https://www.abs.gov.au/statistics/labour/earnings-and-working-conditions/employee-earnings-and-hours-australia/latest-release) into csv format so they can be loaded into S3 bucket.

### 2016

In [2]:
df_2016 = pd.read_excel("../data/EEH 2016.xlsx", sheet_name='Table_1', skiprows=4, header=[0,1], index_col=0)

In [3]:
df_2016.head(20)

Males  \
                                                   Average weekly ordinary time cash earnings   
NaN                                                                                         $   
111 Chief executives, general managers and legi...                                     2806.1   
121 Farmers and farm managers                                                             NaN   
131 Advertising and sales managers                                                     2343.8   
132 Business administration managers                                                   2894.3   
133 Construction, distribution and production m...                                     2296.6   
134 Education, health and welfare services mana...                                     2218.4   
135 ICT managers                                                                       2586.8   
139 Miscellaneous specialist managers                                                  2756.6   
141 Accommodation and hospitality managers                                             1360.6   
142 Retail managers                                                                    1548.9   
149 Miscellaneous hospitality, retail and servi...                                     1849.7   
211 Arts professionals                                                                 1580.3   
212 Media professionals                                                                1888.6   
221 Accountants, auditors and company secretaries                                      1722.8   
222 Financial brokers and dealers, and investme...                                     2360.4   
223 Human resource and training professionals                                          1959.2   
224 Information and organisation professionals                                         2089.2   
225 Sales, marketing and public relations profe...                                     2097.6   
231 Air and marine transport professionals                                             2449.5   

                                                                                       \
                                                   Average weekly total cash earnings   
NaN                                                                                 $   
111 Chief executives, general managers and legi...                             2806.1   
121 Farmers and farm managers                                                     NaN   
131 Advertising and sales managers                                             2348.3   
132 Business administration managers                                           2894.3   
133 Construction, distribution and production m...                               2362   
134 Education, health and welfare services mana...                             2218.4   
135 ICT managers                                                               2594.6   
139 Miscellaneous specialist managers                                          2872.5   
141 Accommodation and hospitality managers                                     1360.6   
142 Retail managers                                                            1553.4   
149 Miscellaneous hospitality, retail and servi...                             1880.5   
211 Arts professionals                                                         1580.3   
212 Media professionals                                                        1894.2   
221 Accountants, auditors and company secretaries                              1722.8   
222 Financial brokers and dealers, and investme...                             2360.4   
223 Human resource and training professionals                                  1969.8   
224 Information and organisation professionals                                 2095.1   
225 Sales, marketing and public relations profe...                             2097.6   
231 Air and marine transport professionals                                     2645.

In [4]:
df_2016.columns

MultiIndex([(  'Males',  'Average weekly ordinary time cash earnings'),
            (  'Males',          'Average weekly total cash earnings'),
            (  'Males', 'Average weekly ordinary time hours paid for'),
            (  'Males',         'Average weekly total hours paid for'),
            (  'Males',  'Average hourly ordinary time cash earnings'),
            (  'Males',          'Average hourly total cash earnings'),
            ('Females',  'Average weekly ordinary time cash earnings'),
            ('Females',          'Average weekly total cash earnings'),
            ('Females', 'Average weekly ordinary time hours paid for'),
            ('Females',         'Average weekly total hours paid for'),
            ('Females',  'Average hourly ordinary time cash earnings'),
            ('Females',          'Average hourly total cash earnings'),
            ('Persons',  'Average weekly ordinary time cash earnings'),
            ('Persons',          'Average weekly total cash earn

In [5]:
# Remove first row
df_2016 = df_2016.iloc[1:]

In [6]:
df_2016_long = df_2016.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()

C:\Users\YenChiNguyen\AppData\Local\Temp\ipykernel_34324\4171460239.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_2016_long = df_2016.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()


In [7]:
# Remove last rows
df_2016_long = df_2016_long.iloc[:-3, :]

In [8]:
df_2016_long

,Occupation,Gender,Average weekly ordinary time cash earnings,Average weekly total cash earnings,Average weekly ordinary time hours paid for,Average weekly total hours paid for,Average hourly ordinary time cash earnings,Average hourly total cash earnings
0,"111 Chief executives, general managers and leg...",Females,2334.7,2334.7,35.8,35.8,65.3,65.3
1,"111 Chief executives, general managers and leg...",Males,2806.1,2806.1,38.4,38.4,73,73
2,"111 Chief executives, general managers and leg...",Persons,2666.2,2666.2,37.6,37.6,70.9,70.9
3,121 Farmers and farm managers,Females,0,0,0,0,0,0
4,131 Advertising and sales managers,Females,2329.9,2329.9,38,38,61.3,61.3
...,...,...,...,...,...,...,...,...
268,851 Food preparation assistants,Persons,1084.5,1116.4,40.7,41.5,26.7,26.9
269,891 Freight handlers and shelf fillers,Persons,1023.5,1213.5,37.8,43.6,27.1,27.8
270,899 Miscellaneous labourers,Females,1082.8,1108.8,39.9,40.5,27.1,27.4
271,899 Miscellaneous labourers,Males,1118.6,1252.7,38.6,41,29,30.5


### 2018

In [9]:
df_2018 = pd.read_excel("../data/EEH 2018.xlsx", sheet_name='Table_1', skiprows=4, header=[0,1], index_col=0)

In [10]:
df_2018.head(20)

Males  \
                                                   Average weekly ordinary time cash earnings   
NaN                                                                                         $   
121 Farmers and farm managers                                                          1821.9   
131 Advertising and sales managers                                                     2638.7   
132 Business administration managers                                                   2795.1   
133 Construction, distribution and production m...                                     2905.9   
134 Education, health and welfare services mana...                                     2345.6   
135 ICT managers                                                                       2855.2   
139 Miscellaneous specialist managers                                                  2522.7   
141 Accommodation and hospitality managers                                             1460.6   
142 Retail managers                                                                    1487.2   
149 Miscellaneous hospitality, retail and servi...                                     1924.4   
211 Arts professionals                                                                 1390.8   
212 Media professionals                                                                1971.8   
221 Accountants, auditors and company secretaries                                      1864.5   
222 Financial brokers and dealers, and investme...                                     2908.4   
223 Human resource and training professionals                                            1944   
224 Information and organisation professionals                                         2279.4   
225 Sales, marketing and public relations profe...                                       2288   
231 Air and marine transport professionals                                             2468.4   
232 Architects, designers, planners and surveyors                                      1800.1   

                                                                                       \
                                                   Average weekly total cash earnings   
NaN                                                                                 $   
121 Farmers and farm managers                                                  1821.9   
131 Advertising and sales managers                                             2638.7   
132 Business administration managers                                           2796.6   
133 Construction, distribution and production m...                             2929.1   
134 Education, health and welfare services mana...                               2348   
135 ICT managers                                                               2862.7   
139 Miscellaneous specialist managers                                          2551.6   
141 Accommodation and hospitality managers                                     1496.8   
142 Retail managers                                                              1488   
149 Miscellaneous hospitality, retail and servi...                             1946.5   
211 Arts professionals                                                         1390.8   
212 Media professionals                                                        2007.5   
221 Accountants, auditors and company secretaries                              1867.5   
222 Financial brokers and dealers, and investme...                             2908.4   
223 Human resource and training professionals                                    1962   
224 Information and organisation professionals                                 2287.9   
225 Sales, marketing and public relations profe...                               2288   
231 Air and marine transport professionals                                       2552   
232 Architects, designers, planners and surveyors                              1817.

In [11]:
df_2018.columns

MultiIndex([(  'Males',  'Average weekly ordinary time cash earnings'),
            (  'Males',          'Average weekly total cash earnings'),
            (  'Males', 'Average weekly ordinary time hours paid for'),
            (  'Males',         'Average weekly total hours paid for'),
            (  'Males',  'Average hourly ordinary time cash earnings'),
            (  'Males',          'Average hourly total cash earnings'),
            ('Females',  'Average weekly ordinary time cash earnings'),
            ('Females',          'Average weekly total cash earnings'),
            ('Females', 'Average weekly ordinary time hours paid for'),
            ('Females',         'Average weekly total hours paid for'),
            ('Females',  'Average hourly ordinary time cash earnings'),
            ('Females',          'Average hourly total cash earnings'),
            ('Persons',  'Average weekly ordinary time cash earnings'),
            ('Persons',          'Average weekly total cash earn

In [12]:
# Remove first row
df_2018 = df_2018.iloc[1:]

In [13]:
df_2018_long = df_2018.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()

C:\Users\YenChiNguyen\AppData\Local\Temp\ipykernel_34324\4150437707.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_2018_long = df_2018.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()


In [14]:
# Remove last rows
df_2018_long = df_2018_long.iloc[:-3, :]

In [15]:
df_2018_long

,Occupation,Gender,Average weekly ordinary time cash earnings,Average weekly total cash earnings,Average weekly ordinary time hours paid for,Average weekly total hours paid for,Average hourly ordinary time cash earnings,Average hourly total cash earnings
0,121 Farmers and farm managers,Females,2201.9,2201.9,38.4,38.4,57.3,57.3
1,121 Farmers and farm managers,Males,1821.9,1821.9,39.6,39.6,46,46
2,121 Farmers and farm managers,Persons,2098.4,2098.4,38.8,38.8,54.1,54.1
3,131 Advertising and sales managers,Females,2262.9,2262.9,38.3,38.3,59,59
4,131 Advertising and sales managers,Males,2638.7,2638.7,38.2,38.2,69.1,69.1
...,...,...,...,...,...,...,...,...
273,891 Freight handlers and shelf fillers,Males,1172.9,1338.6,37.8,41.6,31.1,32.2
274,891 Freight handlers and shelf fillers,Persons,1159.6,1303.5,38,41.3,30.5,31.6
275,899 Miscellaneous labourers,Females,1137.3,1484.1,37.2,41.1,30.6,36.1
276,899 Miscellaneous labourers,Males,1162.2,1291.2,37.9,40.4,30.6,31.9


### 2021

In [16]:
df_2021 = pd.read_excel("../data/EEH 2021.xlsx", sheet_name='Table_1', skiprows=4, header=[0,1], index_col=0)

In [17]:
df_2021.head(20)

Males  \
                                                   Average weekly ordinary time cash earnings   
NaN                                                                                         $   
111 Chief executives and managing directors                                               NaN   
121 Farmers and farm managers                                                             NaN   
131 Advertising and sales managers                                                       2971   
132 Business administration managers                                                     3628   
133 Construction, distribution and production m...                                       2973   
134 Education, health and welfare services mana...                                     2368.5   
135 ICT managers                                                                       3218.9   
139 Miscellaneous specialist managers                                                  2961.6   
141 Accommodation and hospitality managers                                               1579   
142 Retail managers                                                                    1753.1   
149 Miscellaneous hospitality, retail and servi...                                     2008.4   
211 Arts professionals                                                                 1924.4   
212 Media professionals                                                                1895.3   
221 Accountants, auditors and company secretaries                                      2053.6   
222 Financial brokers and dealers, and investme...                                     3404.5   
223 Human resource and training professionals                                          1950.8   
224 Information and organisation professionals                                         2334.9   
225 Sales, marketing and public relations profe...                                     2377.4   
231 Air and marine transport professionals                                             3414.9   

                                                                                       \
                                                   Average weekly total cash earnings   
NaN                                                                                 $   
111 Chief executives and managing directors                                       NaN   
121 Farmers and farm managers                                                     NaN   
131 Advertising and sales managers                                             2971.7   
132 Business administration managers                                             3628   
133 Construction, distribution and production m...                             3067.9   
134 Education, health and welfare services mana...                             2373.7   
135 ICT managers                                                               3218.9   
139 Miscellaneous specialist managers                                          3032.8   
141 Accommodation and hospitality managers                                       1579   
142 Retail managers                                                            1778.9   
149 Miscellaneous hospitality, retail and servi...                             2029.6   
211 Arts professionals                                                         2114.9   
212 Media professionals                                                        1915.1   
221 Accountants, auditors and company secretaries                              2056.3   
222 Financial brokers and dealers, and investme...                             3416.5   
223 Human resource and training professionals                                  1986.6   
224 Information and organisation professionals                                 2340.5   
225 Sales, marketing and public relations profe...                             2383.1   
231 Air and marine transport professionals                                     3445.

In [18]:
# Remove first row
df_2021 = df_2021.iloc[1:]

#TODO: Need to check if you want to keep occupations where there are only null values in the future. df.stack(level=0, dropna=False) e.g. see below occupation 111

In [19]:
df_2021_long = df_2021.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()

C:\Users\YenChiNguyen\AppData\Local\Temp\ipykernel_34324\2510893075.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_2021_long = df_2021.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()


In [20]:
# Remove last rows
df_2021_long = df_2021_long.iloc[:-3, :]

In [21]:
df_2021_long

,Occupation,Gender,Average weekly ordinary time cash earnings,Average weekly total cash earnings,Average weekly ordinary time hours paid for,Average weekly total hours paid for,Average hourly ordinary time cash earnings,Average hourly total cash earnings
0,121 Farmers and farm managers,Persons,1858.6,1858.6,38.7,38.7,48.1,48.1
1,131 Advertising and sales managers,Females,2803.9,2803.9,38.1,38.1,73.6,73.6
2,131 Advertising and sales managers,Males,2971,2971.7,38.2,38.2,77.8,77.8
3,131 Advertising and sales managers,Persons,2906.4,2906.8,38.1,38.2,76.2,76.2
4,132 Business administration managers,Females,2773.4,2773.4,37.3,37.3,74.3,74.3
...,...,...,...,...,...,...,...,...
263,891 Freight handlers and shelf fillers,Males,1392.2,1576.4,38.5,41.9,36.2,37.6
264,891 Freight handlers and shelf fillers,Persons,1357.8,1533.2,38.6,41.9,35.1,36.6
265,899 Miscellaneous labourers,Females,1238.6,1421.5,38.1,41.8,32.5,34
266,899 Miscellaneous labourers,Males,1259.1,1372.1,38.7,40.9,32.5,33.6


In [22]:
df_2021_long[df_2021_long['Occupation']=='111 Chief executives and managing directors']

,Occupation,Gender,Average weekly ordinary time cash earnings,Average weekly total cash earnings,Average weekly ordinary time hours paid for,Average weekly total hours paid for,Average hourly ordinary time cash earnings,Average hourly total cash earnings


### 2023

In [23]:
df_2023 = pd.read_excel("../data/EEH 2023.xlsx", sheet_name='Table_1', skiprows=4, header=[0,1], index_col=0)

In [24]:
df_2023.head(20)

Males  \
                                                   Average weekly ordinary time cash earnings   
NaN                                                                                         $   
111 Chief executives, general managers and legi...                                        NaN   
121 Farmers and farm managers                                                             NaN   
131 Advertising, public relations and sales man...                                     2829.9   
132 Business administration managers                                                   3322.1   
133 Construction, distribution and production m...                                       3200   
134 Education, health and welfare services mana...                                     2548.4   
135 ICT managers                                                                       3626.6   
139 Miscellaneous specialist managers                                                  3130.3   
141 Accommodation and hospitality managers                                             1569.1   
142 Retail managers                                                                    1895.2   
149 Miscellaneous hospitality, retail and servi...                                       2240   
211 Arts professionals                                                                 1639.6   
212 Media professionals                                                                2100.6   
221 Accountants, auditors and company secretaries                                      2058.2   
222 Financial brokers and dealers, and investme...                                     2669.3   
223 Human resource and training professionals                                          2327.3   
224 Information and organisation professionals                                           2421   
225 Sales, marketing and public relations profe...                                     2462.2   
231 Air and marine transport professionals                                             2742.9   

                                                                                       \
                                                   Average weekly total cash earnings   
NaN                                                                                 $   
111 Chief executives, general managers and legi...                                NaN   
121 Farmers and farm managers                                                     NaN   
131 Advertising, public relations and sales man...                             2829.9   
132 Business administration managers                                           3337.6   
133 Construction, distribution and production m...                             3239.4   
134 Education, health and welfare services mana...                             2548.9   
135 ICT managers                                                               3626.6   
139 Miscellaneous specialist managers                                          3166.1   
141 Accommodation and hospitality managers                                     1569.9   
142 Retail managers                                                              1932   
149 Miscellaneous hospitality, retail and servi...                             2266.3   
211 Arts professionals                                                         1641.2   
212 Media professionals                                                        2129.7   
221 Accountants, auditors and company secretaries                              2070.6   
222 Financial brokers and dealers, and investme...                             2669.4   
223 Human resource and training professionals                                  2333.8   
224 Information and organisation professionals                                 2428.2   
225 Sales, marketing and public relations profe...                             2467.2   
231 Air and marine transport professionals                                     2848.

In [25]:
df_2023.columns

MultiIndex([(  'Males',  'Average weekly ordinary time cash earnings'),
            (  'Males',          'Average weekly total cash earnings'),
            (  'Males', 'Average weekly ordinary time hours paid for'),
            (  'Males',         'Average weekly total hours paid for'),
            (  'Males',  'Average hourly ordinary time cash earnings'),
            (  'Males',          'Average hourly total cash earnings'),
            ('Females',  'Average weekly ordinary time cash earnings'),
            ('Females',          'Average weekly total cash earnings'),
            ('Females', 'Average weekly ordinary time hours paid for'),
            ('Females',         'Average weekly total hours paid for'),
            ('Females',  'Average hourly ordinary time cash earnings'),
            ('Females',          'Average hourly total cash earnings'),
            ('Persons',  'Average weekly ordinary time cash earnings'),
            ('Persons',          'Average weekly total cash earn

In [26]:
# Remove first row
df_2023 = df_2023.iloc[1:]

In [27]:
df_2023_long = df_2023.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()

C:\Users\YenChiNguyen\AppData\Local\Temp\ipykernel_34324\3539781405.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_2023_long = df_2023.stack(level=0).reset_index().rename(columns={"level_1": "Gender", "level_0": "Occupation"}).copy()


In [28]:
# Remove last rows
df_2023_long = df_2023_long.iloc[:-3, :]

In [29]:
df_2023_long

,Occupation,Gender,Average weekly ordinary time cash earnings,Average weekly total cash earnings,Average weekly ordinary time hours paid for,Average weekly total hours paid for,Average hourly ordinary time cash earnings,Average hourly total cash earnings
0,121 Farmers and farm managers,Persons,1513.3,1513.3,38.4,38.4,39.2,39.2
1,"131 Advertising, public relations and sales ma...",Females,2493.2,2502.1,37.9,38,65.8,65.9
2,"131 Advertising, public relations and sales ma...",Males,2829.9,2829.9,38.2,38.2,73.9,73.9
3,"131 Advertising, public relations and sales ma...",Persons,2705.7,2709,38.1,38.1,70.9,71
4,132 Business administration managers,Females,2961.2,2961.4,37.8,37.8,78.2,78.2
...,...,...,...,...,...,...,...,...
274,891 Freight handlers and shelf fillers,Males,1299.3,1460.5,38.2,41.7,34.3,35.2
275,891 Freight handlers and shelf fillers,Persons,1304.9,1454.5,38.4,41.6,34.2,35.1
276,899 Miscellaneous labourers,Females,1227.3,1387.5,37.1,40,33.3,34.5
277,899 Miscellaneous labourers,Males,1340.4,1492.5,38.6,41.5,34.7,35.6


### Export to csv (these files are to be loaded to S3 buckets)

In [30]:
dfs_map = {
    "2016": df_2016_long,
    "2018": df_2018_long,
    "2021": df_2021_long,
    "2023": df_2023_long,
}

for name, df in dfs_map.items():
    # Create a Year column
    df['Year'] = name

    # Convert str to int
    df['Year'] = df['Year'].astype(int)

In [31]:
for df in [df_2016_long, df_2018_long, df_2021_long, df_2023_long]:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 9 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Occupation                                   273 non-null    object
 1   Gender                                       273 non-null    object
 2   Average weekly ordinary time cash earnings   273 non-null    object
 3   Average weekly total cash earnings           273 non-null    object
 4   Average weekly ordinary time hours paid for  273 non-null    object
 5   Average weekly total hours paid for          272 non-null    object
 6   Average hourly ordinary time cash earnings   273 non-null    object
 7   Average hourly total cash earnings           272 non-null    object
 8   Year                                         273 non-null    int32 
dtypes: int32(1), object(8)
memory usage: 18.3+ KB
None
<class 'pandas.core.frame.DataFrame'>

In [34]:
dfs = {
    "_2016_s3": df_2016_long,
    "_2018_s3": df_2018_long,
    "_2021_s3": df_2021_long,
    "_2023_s3": df_2023_long,
}

for name, df in dfs.items():
    df.to_csv(f"../data/S3/EEH{name}.csv", index=False)

## Read employment file

In [21]:
df = pd.read_excel("../data/national_occupation_trend_-_august_2025.xlsx", sheet_name='Table_1', skiprows=7)

In [22]:
df.head()

,ANZSCO Level,NFD Indicator 5,Code (Text),ANZSCO Title,COVID Adjusted 3,Skill Level 4,Trend Data for Aug 2025 ('000) ¹,Qtr Change to Aug 2025 ('000) ¹,Qtr Change to Aug 2025 (%) ¹,1 Year Change to Aug 2025 ('000) ¹,...,10 Year Change to Aug 2025 (%) ¹,Share of National Emp (%) 1,Share of Level 1 Group (%) 1,"Female Share, 2024, 4 Quarter Avg (%) ²","Part Time Share, 2024, 4 Quarter Avg (%) ²","Female Part Time Share, 2024, 4 Quarter Avg (%) ²","Male Part Time Share, 2024, 4 Quarter Avg (%) ²",Median Age 2024 (ABS Customised Report) ²,Median Female Age 2024 (ABS Customised Report) ²,Median Male Age 2024 (ABS Customised Report) ²
0,1,N,1,Managers,-,-,1866.720999,9.555297,0.514510,26.038196,...,22.466084,12.737735,-,41,14,21,9,45,43,46
1,2,Y,10,Managers nfd,-,-,5.632010,0.296963,5.566271,1.218784,...,-86.215867,<0.1,0.298971,-,-,-,-,-,-,-
2,3,Y,100,Managers nfd,-,-,5.632010,0.296963,5.566271,1.218784,...,-86.215867,<0.1,0.296307,-,-,-,-,-,-,-
3,4,Y,1000,Managers nfd,-,1,5.632010,0.296963,5.566271,1.218784,...,-86.215867,<0.1,0.295456,-,-,-,-,-,-,-
4,2,N,11,"Chief Executives, General Managers and Legisla...",-,-,130.797606,0.314260,0.240843,1.544866,...,19.631557,0.890325,6.943294,34,13,19,10,49,48,50


In [23]:
df.tail()

,ANZSCO Level,NFD Indicator 5,Code (Text),ANZSCO Title,COVID Adjusted 3,Skill Level 4,Trend Data for Aug 2025 ('000) ¹,Qtr Change to Aug 2025 ('000) ¹,Qtr Change to Aug 2025 (%) ¹,1 Year Change to Aug 2025 ('000) ¹,...,10 Year Change to Aug 2025 (%) ¹,Share of National Emp (%) 1,Share of Level 1 Group (%) 1,"Female Share, 2024, 4 Quarter Avg (%) ²","Part Time Share, 2024, 4 Quarter Avg (%) ²","Female Part Time Share, 2024, 4 Quarter Avg (%) ²","Male Part Time Share, 2024, 4 Quarter Avg (%) ²",Median Age 2024 (ABS Customised Report) ²,Median Female Age 2024 (ABS Customised Report) ²,Median Male Age 2024 (ABS Customised Report) ²
667,9,N,SKI1,Skill Level 1 Occupations,-,-,5210.433119,60.221505,1.169302,193.179133,...,41.700080,35.492932,-,-,-,-,-,-,-,-
668,9,N,SKI2,Skill Level 2 Occupations,Y,-,1804.487858,10.054959,0.560342,37.833079,...,22.930486,12.291985,-,-,-,-,-,-,-,-
669,9,N,SKI3,Skill Level 3 Occupations,-,-,2134.052550,-8.217304,-0.383579,1.135073,...,9.905303,14.536945,-,-,-,-,-,-,-,-
670,9,N,SKI4,Skill Level 4 Occupations,Y,-,3556.538779,6.799475,0.191549,32.423030,...,27.207138,24.226774,-,-,-,-,-,-,-,-
671,9,N,SKI5,Skill Level 5 Occupations,Y,-,1974.686990,-4.175958,-0.211028,-12.736848,...,2.853650,13.451364,-,-,-,-,-,-,-,-


In [24]:
df.columns

Index(['ANZSCO Level', 'NFD Indicator 5', 'Code (Text)', 'ANZSCO Title',
       'COVID Adjusted 3', 'Skill Level 4', 'Trend Data for Aug 2025 ('000) ¹',
       'Qtr Change to Aug 2025 ('000) ¹', 'Qtr Change to Aug 2025 (%) ¹',
       '1 Year  Change to Aug 2025 ('000) ¹',
       '1 Year  Change to Aug 2025 (%) ¹',
       '5 Year  Change to Aug 2025 ('000) ¹',
       '5 Year  Change to Aug 2025 (%) ¹',
       '10 Year  Change to Aug 2025 ('000) ¹',
       '10 Year  Change to Aug 2025 (%) ¹', 'Share of National Emp (%) 1',
       'Share of Level 1 Group (%) 1',
       'Female Share, 2024, 4 Quarter Avg (%) ²',
       'Part Time Share, 2024, 4 Quarter Avg (%) ²',
       'Female Part Time Share, 2024, 4 Quarter Avg (%) ²',
       'Male Part Time Share, 2024, 4 Quarter Avg (%) ²',
       'Median Age 2024 (ABS Customised Report) ²',
       'Median Female Age 2024 (ABS Customised Report) ²',
       'Median Male Age 2024 (ABS Customised Report) ²'],
      dtype='object')

In [25]:
from unidecode import unidecode

df.columns = (
    df.columns
      .map(unidecode)
      .str.replace(r"\s*\(.*?\)", "", regex=True) # remove parentheses content 
      .str.replace(r"[^\w\s]", "", regex=True)  # remove special chars
)

In [26]:
# The superscripts are not removed with the code above but we'll leave it for now
df.columns

Index(['ANZSCO Level', 'NFD Indicator 5', 'Code', 'ANZSCO Title',
       'COVID Adjusted 3', 'Skill Level 4', 'Trend Data for Aug 2025 1',
       'Qtr Change to Aug 2025 1', 'Qtr Change to Aug 2025 1',
       '1 Year  Change to Aug 2025 1', '1 Year  Change to Aug 2025 1',
       '5 Year  Change to Aug 2025 1', '5 Year  Change to Aug 2025 1',
       '10 Year  Change to Aug 2025 1', '10 Year  Change to Aug 2025 1',
       'Share of National Emp 1', 'Share of Level 1 Group 1',
       'Female Share 2024 4 Quarter Avg 2',
       'Part Time Share 2024 4 Quarter Avg 2',
       'Female Part Time Share 2024 4 Quarter Avg 2',
       'Male Part Time Share 2024 4 Quarter Avg 2', 'Median Age 2024 2',
       'Median Female Age 2024 2', 'Median Male Age 2024 2'],
      dtype='object')

In [28]:
df.columns.str.upper()

Index(['ANZSCO LEVEL', 'NFD INDICATOR 5', 'CODE', 'ANZSCO TITLE',
       'COVID ADJUSTED 3', 'SKILL LEVEL 4', 'TREND DATA FOR AUG 2025 1',
       'QTR CHANGE TO AUG 2025 1', 'QTR CHANGE TO AUG 2025 1',
       '1 YEAR  CHANGE TO AUG 2025 1', '1 YEAR  CHANGE TO AUG 2025 1',
       '5 YEAR  CHANGE TO AUG 2025 1', '5 YEAR  CHANGE TO AUG 2025 1',
       '10 YEAR  CHANGE TO AUG 2025 1', '10 YEAR  CHANGE TO AUG 2025 1',
       'SHARE OF NATIONAL EMP 1', 'SHARE OF LEVEL 1 GROUP 1',
       'FEMALE SHARE 2024 4 QUARTER AVG 2',
       'PART TIME SHARE 2024 4 QUARTER AVG 2',
       'FEMALE PART TIME SHARE 2024 4 QUARTER AVG 2',
       'MALE PART TIME SHARE 2024 4 QUARTER AVG 2', 'MEDIAN AGE 2024 2',
       'MEDIAN FEMALE AGE 2024 2', 'MEDIAN MALE AGE 2024 2'],
      dtype='object')

In [27]:
df.to_csv("../data/S3/employment.csv", index=False)